In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import chi2
from scipy.stats import f
from numpy.linalg import multi_dot
from numpy.linalg import inv
from numpy.linalg import pinv
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
# set format of the output
float_formatter = "{:.4f}".format # keep 4 digits
np.set_printoptions(formatter={'float_kind':float_formatter})

#******************************************************************************
# Define Paths + Read in Data + Clean Data
#******************************************************************************
# define paths of files 
path = os.getcwd() + "/Data"

# path = '/Users/keling/Dropbox/Courses/Keling/aptest_keling/Data'
FFFactors_path = '/factors/FFFactors.txt'
FF25_path = '/FF25/ff25vm.txt'
size10_path = '/size10/sizevm.txt'
industry10_path = '/industry10/ind10vm.txt'
sm25_path = '/SM/25sm.txt'
FF3New_path = '/FF3/ff3.txt'

In [3]:
# read in files
FFFactors = pd.read_csv(path + FFFactors_path, header = None, delim_whitespace=True)
FFFactors.columns = ['time', 'MKT-RF', 'SMB','HML', 'RF']

# transform into matrix form
rmx = FFFactors['MKT-RF'].to_numpy().reshape(-1,1) # market excess return 
rf = FFFactors['RF'].to_numpy().reshape(-1,1) # risk free rate

FF25 = pd.read_csv(path + FF25_path, header = None, delim_whitespace=True)
size10 = pd.read_csv(path + size10_path, header = None, delim_whitespace=True)
industry10 = pd.read_csv(path + industry10_path, header = None, delim_whitespace=True)
sm25 = pd.read_csv(path + sm25_path, header = None, delim_whitespace=True)
FF3 = pd.read_csv(path + FF3New_path, header = None, delim_whitespace=True)
FF3.columns = ['time', 'MKT-RF', 'SMB','HML', 'RF']
# select test assets
# ta = 1: Fama-French 25 portfolios
# ta = 2: 10 size deciles
# ta = 3: 10 industry portfolios
# ta = 4: size and value factors - SMB and HML

ta = 0
if ta ==0:
    testass = sm25.iloc[:,1:26].to_numpy()
if ta ==1:
    testass = FF25.iloc[:,1:26].to_numpy()
if ta ==2:
    testass = size10.iloc[:,10:20].to_numpy()
if ta ==3:
    testass = industry10.iloc[:,1:11].to_numpy()
if ta ==4:
    testass = FFFactors[['SMB','HML']].to_numpy()

# deal with missing values
testass[testass == -99.99] = 0
# check size of test assets: 25 portfolios / 10 size deciles/ 10 industry
n = testass.shape[1]

if ta == 0:
    rmx = FF3['MKT-RF'].to_numpy().reshape(-1,1) # market excess return 
    rf = FF3['RF'].to_numpy().reshape(-1,1) # risk free rate
    smb = FF3['SMB'].to_numpy().reshape(-1,1)
    hml = FF3['HML'].to_numpy().reshape(-1,1)
    ff3 = FF3[['MKT-RF', 'SMB', 'HML']].to_numpy()#Fama French 3 factor
    tax = np.subtract(testass, rf)
elif 0<ta<4:
    tax = np.subtract(testass, rf)
else:
    tax = testass

# Part 1.
## a). Regress each return on the market return and generate a set of alphas.

In [5]:
#%%
#******************************************************************************
# Q1 (a)
#******************************************************************************
print('\n\n\n\n\n')
print('********************OLS TIME SERIES******************************')
#******TIME SERIES REGRESSION*****************
model = LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
X = ff3
Y = tax
regressor = model.fit(X, Y)

# retrieving the coefficient and intercept
b = regressor.coef_ 
a = regressor.intercept_

#******CALCULATE MOMENTS***************** ######%%%%%%%%%%%%%%%
mnxret = np.mean(tax, axis = 0) # mean (excess) test assets returns (dimension: num test assets*1)
mnff3 = np.mean(ff3, axis = 0) # mean return for the factors
capt = len(rf) # length of data
covf = (1/(capt-1))* np.dot((ff3-mnff3).transpose(), (ff3-mnff3)) # sample variance of market excess return
#sigf = np.sqrt(varf) # sample volatility of market excess return
#fsharpe = mnff3/sigf

#*********PLOTS RETURNS AND THE MARKET SHARPE RATIO LINE***************
#plt.plot(b, mnxret, 'o', 1, mnrmx,'kx', [0, 1.5],[0, 1.5*mnrmx],'k-') #return as dot, [1, mnrmx] as the E[rm], line as the SR line
#minr = min(np.append([0], mnxret))
#plt.axis([0, 1.5, minr, 1.2])
#plt.title("OLS Time Series")
#plt.xlabel(r"$\beta$")
#plt.ylabel(r"$E[R^{e}]$")
#plt.show()







********************OLS TIME SERIES******************************


/Users/miao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


## b). Calculate the covariance of the alphas assuming
i). No correlation among the alphas.

ii). No temporal correlation among the alphas.

iii). Temporal correlation at one lag.

iv). Using the Newey-West kernel.

In [6]:
#%%
#******************************************************************************
# Q1 (b)
#******************************************************************************

#*********GET RESIDUALS AND COVARIANCE MATRICES*****************

# residuals (Dimension: capt*1) ##u = Y-Xb'-a, dim: capt*25 for ff25
u = Y - np.dot(X, b.transpose()) - np.multiply(np.asarray(np.ones(capt)).reshape(capt,1), a.transpose())#####%%%%%%%%%%

# Q1 (b) ii: iid through time but may be cross-sectionally correlated
covmat = (1/(capt-1)) * np.dot(u.transpose(), u)  

# Q1 (b) i: assuming residuals are uncorrelated (not a good assumption, just for comparison)
covnoc = np.diag(np.diag(covmat)) #make the diagnoal element a matrix  

# Q1 (b) iii: one lag of autocorrelations;
#This is using the NW(1987) Bartlett weight 2*(1-i/(m+1));                           
cov1l = covmat + (1/(capt-2)) * np.dot(u[:-1,:].transpose(), u[1:,:])
#Alternative is Hansen-Hodrick (see Cochrane p. 210);                     
cov1l = covmat + (2/(capt-2)) * np.dot(u[:-1,:].transpose(), u[1:,:])  

# Q1 (b) iv: Four lags, Newey-West (Bartlett) weights;
cov4l = covmat 
# For how to choose mm, see NW(1994) "Automatic lag selection..." 
mm = 4                      
for i in range(1, mm+1):
    w = 1 - i/(mm+1)
    cov4l = cov4l + 2*w*(1/(capt-1-i))*np.dot(u[:-i,:].transpose(), u[i:,:])  

## c). The code calculates the following test statistics for each of the assumed covariance matrices of alpha above
i). The OLS asymptotic chi-squared statistics and p-values

ii). The GRS finite-sample F statistics and p-values

iii). GMM time series asymptotic (chi-squared) and finite sample (F)

In [13]:
#******************************************************************************
# Q1 (c)
#******************************************************************************

#**********CALCULATE TEST STATISTICS ASSUMING FACTOR RETURNS AND TESTMAT RESIDUALS INDEPENDENT*********; 
#See Cochrane p. 230;
# Q1 (c) i: With errors that are i.i.d over time, homoskedastic and independent of the factors, 
# the asymptotic joint distribution of the intercepts gives the model test statistic Chi-Square

prechi = capt * inv(1 + mnff3.reshape(-1,1).transpose()@inv(covf)@mnff3.reshape(-1,1))
#######%%%%%% E_T(f)'\Omega^{-1}E_T(f) p.231 (12.3)
alphvec = a.reshape(-1,1)

chi_noc = (prechi*alphvec.transpose()@inv(covnoc)@alphvec).item()
chi_reg = (prechi*alphvec.transpose()@inv(covmat)@alphvec).item()
chi_1l = (prechi*alphvec.transpose()@inv(cov1l)@alphvec).item()
chi_4l = (prechi*alphvec.transpose()@inv(cov4l)@alphvec).item()


chis = np.array([chi_noc, chi_reg, chi_1l, chi_4l])
pchi = np.ones(4)-chi2.cdf(chis,n)

# print the result
print('OLS asymptotic test statistics (Chi-Square), assuming factor returns and testmat residuals are independent\n')  
print(chis)
print(pchi)
print('\n')


# See Cochrane p. 230
# Q1 (c) ii: With errors are normally distributed, GRS test gives a multivariate, finite-sample test statistic
pref = ((capt-n-3)/n) * (1/(1+mnff3.reshape(-1,1).transpose()@inv(covf)@mnff3.reshape(-1,1))) 
#######%%%%%%k = 3 for three factor p.231 (12.4)
fs = np.multiply(chis, np.divide(pref, prechi))
pf = np.ones(4) - list(f.cdf(fs,n,capt-n-3))#####%%%% k = 3

# print the result
print('GRS finite-sample test-statistic\n')
print(fs)
print(pf)
print('\n')

OLS asymptotic test statistics (Chi-Square), assuming factor returns and testmat residuals are independent

[500.8556 118.1028 118.9648 115.5317]
[0.0000 0.0000 0.0000 0.0000]


GRS finite-sample test-statistic

[[19.5473 4.6093 4.6429 4.5089]]
[[0.0000 0.0000 0.0000 0.0000]]




In [80]:
#d1 = np.array([1]+list(mnff3)).reshape(1,-1)
#d2 = np.concatenate((mnff3.reshape(-1,1), mnff32), axis = 1)
#d12 = np.concatenate((d1, d2), axis = 0)
#d = - np.kron(d12, np.eye(n))

In [75]:
#d = - np.kron([[1, mnff3], [mnff3, mnff32]],np.eye(n))

In [81]:
#inv(d)

array([[-1.0213, -0.0000, -0.0000, ..., -0.0000, -0.0000, -0.0000],
       [-0.0000, -1.0213, -0.0000, ..., -0.0000, -0.0000, -0.0000],
       [-0.0000, -0.0000, -1.0213, ..., -0.0000, -0.0000, -0.0000],
       ...,
       [-0.0000, -0.0000, -0.0000, ..., -0.0832, -0.0000, -0.0000],
       [-0.0000, -0.0000, -0.0000, ..., -0.0000, -0.0832, -0.0000],
       [-0.0000, -0.0000, -0.0000, ..., -0.0000, -0.0000, -0.0832]])

In [82]:
#np.shape(cmuu_noc)

(50, 50)

In [83]:
#%%
# Q1 (c) iii:
#******************************************************;
#  GMM Time-Series (Allows More General Errors)        ;
#               See p. 234 Cochrane                    ;
#******************************************************;
mnff32 = np.mean(np.square(ff3))

#first displayed formula, top of 234;
d = - np.kron([[1, np.mean(mnff3)], [np.mean(mnff3), mnff32]],np.eye(n))#####%%%%p.234 first equation
u_x = np.multiply(u, np.dot(ff3, np.ones((3,n))))#######%%%%%%%
uu = np.concatenate((u, u_x), axis=1)

cmuu = (1/(capt-1)) * np.dot(uu.transpose(), uu)#No temporal correlation
cmuu_noc = np.diag(np.diag(cmuu))#nocorrelation

# %NW version 1 lag;
c1luu = cmuu + (1/(capt-2)) * np.dot(uu[:-1,:].transpose(), uu[1:,:]) 
# %Hansen-Hodrick version 1 lag;
c1luu = cmuu + (2/(capt-2)) * np.dot(uu[:-1,:].transpose(), uu[1:,:])   

# %NW general
c4luu = cmuu;   
mm = 4;
for i in range(1, mm+1):
    w = 1 - i/(mm+1);
    c4luu = c4luu + 2*w*(1/(capt-1-i))*np.dot(uu[:-i,:].transpose(), uu[i:,:])  

dinv = inv(d);
vp1 = (1/capt) * dinv@cmuu_noc@dinv.transpose()
vp2 = (1/capt) * dinv@cmuu@dinv.transpose()
vp3 = (1/capt) * dinv@c1luu@dinv.transpose()
vp4 = (1/capt) * dinv@c4luu@dinv.transpose()
##12.7

sdb1 = np.sqrt(np.diag(vp1)).transpose()
sdb2 = np.sqrt(np.diag(vp2)).transpose()
sdb3 = np.sqrt(np.diag(vp3)).transpose()
sdb4 = np.sqrt(np.diag(vp4)).transpose()

wt1 = inv(vp1[0:n, 0:n])
wt2 = inv(vp2[0:n, 0:n])
wt3 = inv(vp3[0:n, 0:n])
wt4 = inv(vp4[0:n, 0:n])
##p.234 last equation

chi2a = (alphvec.transpose()@wt1@alphvec).item()
chi2b = (alphvec.transpose()@wt2@alphvec).item()
chi2c = (alphvec.transpose()@wt3@alphvec).item()
chi2d = (alphvec.transpose()@wt4@alphvec).item()

chis2 = np.array([chi2a,chi2b,chi2c,chi2d])

pchi2 = np.ones(4) - chi2.cdf(chis2,n)

print('GMM Results Time-Series asymptotic \n')
print(chis2)
print(pchi2)

print('\n')

fs2 = np.multiply(chis2, np.divide(pref, prechi))
pf2 = np.ones(4) - list(f.cdf(fs2,n,capt-n-1))

print('GMM Results Time-Series finite sample \n');
print(fs2)
print(pf2)

GMM Results Time-Series asymptotic 

[389.0632 135.9143 143.3972 141.0208]
[0.0000 0.0000 0.0000 0.0000]


GMM Results Time-Series finite sample 

[[15.1843 5.3044 5.5965 5.5037]]
[[0.0000 0.0000 0.0000 0.0000]]


## Part 2.a The cross-sectional approach
Using the betas previously estimated, run cross-sectional regression to estimate
the market price of risk. Enforce the assumption that the zero beta portfolio has
zero excess return.

In [84]:
#%%
#******************************************************************************
# Q2 (a)
#******************************************************************************

#*********CROSS-SECTIONAL REGRESSIONS;**********************************

print('\n\n\n\n\n')
print('********************OLS CROSS SECTIONAL******************************')

#  %See p. 237 of Cochrane;
#%Putting a constant here would allow the zero beta portfolio to be unrestricted;
X = b # beta estimate from time-series regression     
Y = mnxret

model = LinearRegression(fit_intercept=False, normalize=False, copy_X=True, n_jobs=None) #notice no intercept here
regressor = model.fit(X, Y)

# retrieving the coefficient
lam = regressor.coef_ # lambda: risk premia

alphcs = Y.reshape(-1,1) - np.dot(X, lam).reshape(-1, 1) #####Y.reshape(-1,1) - np.multiply(X, lam) # alpha estimation

xxinv = inv(np.dot(X.transpose(), X))
cap = np.eye(n) - multi_dot([X, xxinv, X.transpose()]) # I - X(X'X)^(-1)X'







********************OLS CROSS SECTIONAL******************************


/Users/miao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


In [131]:
np.shape(Y.reshape(-1,1) - np.dot(X, lam).reshape(-1, 1))#- np.multiply(X, lam))

(25, 1)

## Part 2.b 
The code calculates the following test statistics assuming each of the error
structures in 1b

### i) OLS asymptotic chi-squared statistics.

In [87]:
#%%
# Q2 (b) i
#####################################################################################
# variance of lambda estimation and variance of lambda estimation
# assume that residuals are not correlated --> the covariance variance matrix \Lamba in (12.12) and (12.13)
#####################################################################################
# here it seems that the variance covariance matrix of factors is missing from  
# the matlab formula when error and factors are iid
#####################################################################################

# factors and errors are iid through time
#varlam1 = (1/capt) * (xxinv * multi_dot([X.transpose(), covnoc, X]) * xxinv + varf)
varlam1 = (1/capt) * (xxinv @ multi_dot([X.transpose(), covnoc, X]) @ xxinv)
covalph1 = (1/capt) * multi_dot([cap, covnoc, cap]) 

# iid through time but may be cross-sectionally correlated
#varlam2 = (1/capt) * (xxinv * multi_dot([X.transpose(), covmat, X]) * xxinv + varf)
varlam2 = (1/capt) * (xxinv @ multi_dot([X.transpose(), covmat, X]) @ xxinv)
covalph2 = (1/capt) * multi_dot([cap, covmat, cap])

# 1 lag of autocorrelations
#varlam3 = (1/capt) * (xxinv * multi_dot([X.transpose(), cov1l, X]) * xxinv + varf)
varlam3 = (1/capt) * (xxinv @ multi_dot([X.transpose(), cov1l, X]) @ xxinv)
covalph3 = (1/capt) * multi_dot([cap, cov1l, cap])

# 4 lags of autocorrelations
#varlam4 = (1/capt) * (xxinv * multi_dot([X.transpose(), cov4l, X]) * xxinv + varf)
varlam4 = (1/capt) * (xxinv @ multi_dot([X.transpose(), cov4l, X]) @ xxinv)
covalph4 = (1/capt) * multi_dot([cap, cov4l, cap])

sealph1 = np.sqrt(np.diag(covalph1)).transpose()
sealph2 = np.sqrt(np.diag(covalph2)).transpose()
sealph3 = np.sqrt(np.diag(covalph3)).transpose()
sealph4 = np.sqrt(np.diag(covalph4)).transpose()

alphcsp = alphcs.transpose()

# standard error of estimated alpha
print(sealph1)
print(sealph2)
print(sealph3)
print(sealph4)

# standard error of estimated lambda
siglam = np.sqrt([varlam1.item(0, 0), varlam2.item(0, 0), varlam3.item(0, 0), varlam4.item(0, 0)])
print(siglam)

# the Moore-Penrose pseudo inverse of matrix
chi2_1 = (alphcs.transpose()@pinv(covalph1)@alphcs).item()
chi2_2 = (alphcs.transpose()@pinv(covalph2)@alphcs).item()
chi2_3 = (alphcs.transpose()@pinv(covalph3)@alphcs).item()
chi2_4 = (alphcs.transpose()@pinv(covalph4)@alphcs).item()

chivec = np.array([chi2_1, chi2_2, chi2_3, chi2_4])
pchi = 1 - chi2.cdf(chivec, n-3)####k = 3

print('OLS asymptotic test statistics (Chi-Square), assuming factor returns and testmat residuals are independent\n')  
print(chivec)
print(pchi)


#plt.plot(b, mnxret, 'o', 1,mnrmx,'kx', [0, 1.5],[0, 1.5*lam],'k--')
#plt.plot([0, 1.5],[0, 1.5*lam],'k--')
#minr = min(np.append([0], mnxret))
#plt.axis([0, 1.5, minr, 1.2])

#plt.title("OLS Cross Sectional")
#plt.xlabel(r"$\beta$")
#plt.ylabel(r"$E[R^{e}]$")

#plt.show()


[0.1074 0.0831 0.0808 0.0936 0.0939 0.0972 0.0656 0.0603 0.0637 0.0698
 0.1058 0.0633 0.0547 0.0539 0.0674 0.1130 0.0697 0.0553 0.0548 0.0698
 0.1115 0.0717 0.0523 0.0505 0.0679]
[0.0903 0.0643 0.0700 0.0846 0.0777 0.0821 0.0497 0.0584 0.0585 0.0485
 0.0798 0.0451 0.0495 0.0520 0.0443 0.0809 0.0467 0.0476 0.0535 0.0424
 0.0879 0.0514 0.0497 0.0595 0.0584]
[0.0859 0.0649 0.0684 0.0927 0.0716 0.0887 0.0458 0.0524 0.0608 0.0485
 0.0857 0.0413 0.0482 0.0527 0.0458 0.0792 0.0449 0.0491 0.0564 0.0422
 0.0817 0.0469 0.0494 0.0604 0.0624]
[0.0926 0.0668 0.0694 0.0967 0.0788 0.0885 0.0471 0.0518 0.0605 0.0485
 0.0853 0.0408 0.0478 0.0548 0.0466 0.0834 0.0458 0.0486 0.0579 0.0451
 0.0865 0.0507 0.0497 0.0585 0.0632]
[0.0249 0.0279 0.0281 0.0266]
OLS asymptotic test statistics (Chi-Square), assuming factor returns and testmat residuals are independent

[235.4260 111.2438 107.3279 104.5899]
[0.0000 0.0000 0.0000 0.0000]


### ii) OLS with the Shanken correction for estimation error in the first-stage estimated betas

In [89]:
#%%
# Q2 (b) ii
#*************************************************************************************;
# Shanken Correction, addressing the fact that our beta is estimated using time-series regression 
# See p. 239 of Cochrane, equation (12.19)
print('\n\n\n');
print('************************** Shanken Correction OLS *************************** ')

mc = 1 + lam.transpose()@inv(covf)@lam #######%%%%%%%change inv(np.asarray(varrmx).reshape(1,1)) to inv(covf)
ac = covf#######%%%%%%%change varrmx to covf

varlam1s = (1/capt) * xxinv * multi_dot([X.transpose(), covnoc, X]) * xxinv * mc + (1/capt)*ac
varlam2s = (1/capt) * xxinv * multi_dot([X.transpose(), covmat, X]) * xxinv * mc + (1/capt)*ac
varlam3s = (1/capt) * xxinv * multi_dot([X.transpose(), cov1l, X]) * xxinv * mc + (1/capt)*ac
varlam4s = (1/capt) * xxinv * multi_dot([X.transpose(), cov4l, X]) * xxinv * mc + (1/capt)*ac

siglams = np.sqrt([varlam1s, varlam2s, varlam3s, varlam4s])
print(siglams)

covalph1s = covalph1*mc
covalph2s = covalph2*mc
covalph3s = covalph3*mc
covalph4s = covalph4*mc

sealph1s = np.sqrt(np.diag(covalph1s)).transpose()
sealph2s = np.sqrt(np.diag(covalph2s)).transpose()
sealph3s = np.sqrt(np.diag(covalph3s)).transpose()
sealph4s = np.sqrt(np.diag(covalph4s)).transpose()

print(sealph1s)
print(sealph2s)
print(sealph3s)
print(sealph4s)

chi1s = (alphcs.transpose()@pinv(covalph1s)@alphcs).item() #(1,1) array
chi2s = (alphcs.transpose()@pinv(covalph2s)@alphcs).item()
chi3s = (alphcs.transpose()@pinv(covalph3s)@alphcs).item()
chi4s = (alphcs.transpose()@pinv(covalph4s)@alphcs).item()

print('\n');
print('OLS with Shanken correction for estimatin error in the first-stage estimated betas\n')  
chivecs = np.array([chi1s, chi2s, chi3s, chi4s])
pchis = 1 - chi2.cdf(chivecs, n-3)####%%%% k = 3

print(chivecs)
print(pchis)


print('\n\n\n\n\n');





************************** Shanken Correction OLS *************************** 
[[[0.1660 0.0702 0.0753]
  [0.0702 0.1292 0.0779]
  [0.0753 0.0779 0.2245]]

 [[0.1795 0.0735 0.1056]
  [0.0735 0.1665 0.1328]
  [0.1056 0.1328 0.4392]]

 [[0.1807 0.0736 0.1063]
  [0.0734 0.1652 0.1279]
  [0.1067 0.1330 0.4367]]

 [[0.1820 0.0745 0.1098]
  [0.0738 0.1732 0.1354]
  [0.1088 0.1401 0.4525]]]
[0.1161 0.0898 0.0874 0.1012 0.1015 0.1051 0.0709 0.0652 0.0689 0.0755
 0.1144 0.0684 0.0591 0.0582 0.0729 0.1221 0.0753 0.0598 0.0592 0.0755
 0.1205 0.0775 0.0565 0.0546 0.0734]
[0.0977 0.0695 0.0757 0.0914 0.0839 0.0887 0.0537 0.0632 0.0633 0.0524
 0.0863 0.0488 0.0535 0.0562 0.0479 0.0874 0.0505 0.0515 0.0578 0.0458
 0.0950 0.0556 0.0537 0.0643 0.0631]
[0.0928 0.0701 0.0740 0.1002 0.0773 0.0958 0.0495 0.0567 0.0657 0.0525
 0.0927 0.0447 0.0521 0.0570 0.0495 0.0856 0.0486 0.0531 0.0609 0.0457
 0.0883 0.0507 0.0534 0.0653 0.0674]
[0.1001 0.0722 0.0750 0.1045 0.0851 0.0957 0.0510 0.0560 0.0653 0.0524
 

### iii) GLS with and without the Shanken correction

In [90]:
#%%
#**************************
#    GLS Cross-Section    ;
#**************************
#See Cochrane p.238, 
#Betas estimated more efficiently; 

###############################################################################
#matlab code also misses the covariance variance matrix of factors
###############################################################################
print('***********GLS Cross-Section (Assume iid)********* ')

lamg = inv(X.transpose()@inv(covmat)@X)@X.transpose()@inv(covmat)@Y # (12.15)
print(lamg)

agls = Y.reshape(-1,1) - np.dot(X, lamg).reshape(-1,1)########%%%%%changed multiply to dot
#%%
#plt.plot(b, mnxret, 'o', 1,mnrmx,'kx',[0, 1.5],[0, 1.5*lamg.item()],'k:')
#plt.title("GLS Cross Sectional")
#plt.xlabel(r"$\beta$")
#plt.ylabel(r"$E[R^{e}]$")
#plt.show()
#%%

varlamg = (1/capt) * inv(X.transpose()@inv(covmat)@X) + (1/capt) * covf # (12.16) ######%%%%%covf
#varlamg = (1/capt) * inv(X.transpose()@inv(covmat)@X)
cova = (1/capt) * (covmat - X@inv(X.transpose()@inv(covmat)@X)@X.transpose()) #(12.17)

selamg = np.sqrt(np.diag(varlamg)).transpose()
sea = np.sqrt(np.diag(cova)).transpose()

print(selamg)
print(sea)
#%%
aglsp = agls.transpose()

chigls = capt * agls.transpose()@inv(covmat)@agls # (12.18)
pchigls = 1 - chi2.cdf(chigls,n-1);

print('\n');
print('GLS without Shanken correction\n')  
print(chigls)
print(pchigls)

print('\n\n\n\n\n')

***********GLS Cross-Section (Assume iid)********* 
[0.7089 0.2846 0.1231]
[0.1595 0.1043 0.1687]
[0.1103 0.0635 0.0624 0.0894 0.0993 0.0945 0.0502 0.0493 0.0579 0.0610
 0.1039 0.0510 0.0447 0.0503 0.0559 0.1160 0.0612 0.0480 0.0513 0.0634
 0.1200 0.0640 0.0389 0.0419 0.0638]


GLS without Shanken correction

[[111.2438]]
[[0.0000]]








In [91]:
#%%
#****************************************************
#GLS Cross-Section Shanken Correction;  
#****************************************************
#See Cochrane p. 239
print('***********GLS Cross-Section Shanken Correction (Assume iid)********* ')

# varlamgs is also wrong in matlab code
varlamgs = (1/capt) * inv(X.transpose()@inv(covmat)@X)*mc + (1/capt)*ac #(12.19)
covas = cova*mc

selamgs = np.sqrt(np.diag(varlamgs)).transpose()
seas = np.sqrt(np.diag(covas)).transpose()

print(selamgs)
print(seas)
#%%
chiglss = capt * agls.transpose()@inv(covmat)@agls * mc #(12.21)
pchiglss = 1 - chi2.cdf(chiglss,n-1)

print('\n');
print('GLS with Shanken correction\n')  
print(chiglss)
print(pchiglss)

***********GLS Cross-Section Shanken Correction (Assume iid)********* 
[0.1597 0.1060 0.1772]
[0.1192 0.0686 0.0674 0.0966 0.1073 0.1021 0.0543 0.0533 0.0626 0.0659
 0.1123 0.0551 0.0483 0.0544 0.0605 0.1254 0.0662 0.0519 0.0554 0.0686
 0.1297 0.0692 0.0420 0.0453 0.0689]


GLS with Shanken correction

[[129.9536]]
[[0.0000]]


In [249]:
#*****************************************************;
#OLS Cross-Section by GMM;
#See Cochrane p. 241 - 243;
#******************************************************;

print('\n\n\n')
print('OLS Cross-Section by GMM (general errors)')

X = b

a_1 = np.concatenate((np.eye(2*n), np.zeros((2*n,n))), axis =1) 
a_2 = np.concatenate((np.zeros((3,2*n)), X.transpose()), axis = 1)########%%%%%%%%
a = np.concatenate((a_1, a_2), axis =0) 

d_1 = np.concatenate((np.eye(n), np.eye(n)*mnrmx, np.zeros((n,1))), axis =1) 
d_2 = np.concatenate((np.eye(n)*mnrmx, np.eye(n)*mnrmx2, np.zeros((n,1))), axis =1) 
d_3 = np.concatenate((np.zeros((n,n)), lam*np.eye(n), X), axis =1) 
d = -np.concatenate((d_1, d_2, d_3), axis =0)
  
uu = np.concatenate((u, np.multiply(u, rmx*np.ones((1,n))), tax-np.dot(np.ones((capt,1)), X.transpose())*lam), axis = 1)
cmuu = (1/(capt-1)) * np.dot(uu.transpose(), uu)
cmuu_noc = np.diag(np.diag(cmuu))

c1luu = cmuu + (1/(capt-2)) * np.dot(uu[:-1,:].transpose(), uu[1:,:])
c4luu = cmuu
for i in range(2, 5):
  w = 1 - i/5
  c4luu = c4luu + 2*w*(1/(capt-1-i))*np.dot(uu[0:-i,:].transpose(), uu[i:,:])

invad = inv(np.dot(a,d))
varb2 = (1/capt)* multi_dot([invad, a, cmuu, a.transpose(), invad])

premom = np.subtract(np.eye(3*n), multi_dot([d, invad, a]))
varmom1 = (1/capt)*multi_dot([premom, cmuu_noc, premom.transpose()])
varmom2 = (1/capt)*multi_dot([premom, cmuu, premom.transpose()])
varmom3 = (1/capt)*multi_dot([premom, c1luu, premom.transpose()])
varmom4 = (1/capt)*multi_dot([premom, c4luu, premom.transpose()])

alph = np.mean(uu[:,2*n:], axis = 0)
chi21 = multi_dot([alph, pinv(varmom1[2*n:,2*n:]), alph.transpose()])
chi22 = multi_dot([alph, pinv(varmom2[2*n:,2*n:]), alph.transpose()])
chi23 = multi_dot([alph, pinv(varmom3[2*n:,2*n:]), alph.transpose()])
chi24 = multi_dot([alph, pinv(varmom4[2*n:,2*n:]), alph.transpose()])
chi2og = np.array([chi21, chi22, chi23, chi24])

pchi2og = 1 - chi2.cdf(chi2og,n-1);

print('\n');
print(chi2og)
print(pchi2og)





OLS Cross-Section by GMM (general errors)


ValueError: operands could not be broadcast together with shapes (3,) (25,25) 

In [222]:
#%%
#*************************************;
#GLS Cross-Section by GMM;
print('\n\n\n');
print('GLS Cross-Section by GMM (general errors)');

a_1 = np.concatenate((np.eye(2*n), np.zeros((2*n,n))), axis =1) 
a_2 = np.concatenate((np.zeros((1, 2*n)), np.dot(X.transpose(), inv(covmat))), axis =1) 

a = np.concatenate((a_1, a_2), axis=0)

invadg = inv(np.dot(a, d))
varb2g = (1/capt)*multi_dot([invadg, a, cmuu, a.transpose(), invadg])

premomg = np.eye(3*n)-multi_dot([d, invadg, a])
varmom1g = (1/capt)*multi_dot([premomg, cmuu_noc, premomg.transpose()])
varmom2g = (1/capt)*multi_dot([premomg, cmuu, premomg.transpose()])
varmom3g = (1/capt)*multi_dot([premomg, c1luu, premomg.transpose()])
varmom4g = (1/capt)*multi_dot([premomg, c4luu, premomg.transpose()])

alph = np.mean(uu[:,2*n:], axis = 0)
chi21g = multi_dot([alph, pinv(varmom1g[2*n:,2*n:]), alph.transpose()])
chi22g = multi_dot([alph, pinv(varmom2g[2*n:,2*n:]), alph.transpose()])
chi23g = multi_dot([alph, pinv(varmom3g[2*n:,2*n:]), alph.transpose()])
chi24g = multi_dot([alph, pinv(varmom4g[2*n:,2*n:]), alph.transpose()])
chi2g = np.array([chi21g, chi22g, chi23g, chi24g])

pchi2g = 1 - chi2.cdf(chi2g,n-1)

print('\n');
print(chi2g)
print(pchi2g)





GLS Cross-Section by GMM (general errors)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 1 and the array at index 1 has size 3